**Reconocimiento de Patrones en Imágenes**

Se cuenta con una base de datos de 10.000 patches a color de 64x64 pixeles, correspondientes a porciones de paredes que han sido y que no han sido rayadas, distribuidas 50-50%, es decir 5.000 patches pertencientes a la clase 1 (rayas), y 5.000 patches pertenecientes a la clase 0 (no-rayas).

Cada uno de estos patches cubre aproximadamente una superficie de 30cm x 30cm de la pared.

Se debe disenar un clasificador que funcione con un maximo de 50 caraceristicas, para esto se deben sacar al menos 200 caracteristica y a partir de tecnicas se seleccion o transformacion de caracteristicas se le debe proporcionar al clasificador un maximo de 50 caracteristicas. 

El clasificador a emplear es un KNN de tres vecinos.

In [1]:
import cv2
import sys
import numpy as np
from os import listdir
import fnmatch, os

import pandas as pd
import warnings
import tqdm
from PIL import Image
from seaborn import heatmap
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from itertools import combinations
from pybalu.feature_extraction import lbp_features, hog_features, gabor_features, haralick_features

In [2]:
from imageio import imread as _imread

def imread(filename, *, normalize=False, flatten=False):
    img = _imread(filename)
    if flatten:
        img = img @ [0.299, 0.587, 0.114]
    if normalize:
        return (img / 255)
    return img.astype(_np.uint8)

In [3]:
# Esta es la funcion de Pybalu, permitida usarla segun enunciado
def clean(features, show=False):
    n_features = features.shape[1]
    ip = np.ones(n_features, dtype=int)

    # cleaning correlated features
    warnings.filterwarnings('ignore')
    C = np.abs(np.corrcoef(features, rowvar=False))
    idxs = np.vstack(np.where(C > .99))
    
    # remove pairs of same feature ( feature i will have a correlation of 1 whit itself )
    idxs = idxs[:, idxs[0,:] != idxs[1,:]]
    
    # remove correlated features
    if idxs.size > 0:
        ip[np.max(idxs, 0)] = 0
    
    # remove constant features
    s = features.std(axis=0, ddof=1)
    ip[s < 1e-8] = 0
    p = np.where(ip.astype(bool))[0]

    if show:
        print(f'Clean: number of features reduced from {n_features} to {p.size}.')

    return p

In [ ]:
def jfisher(features, classification, p=None): 
    m = features.shape[1]
    
    norm = classification.ravel() - classification.min()
    max_class = norm.max() + 1
    
    if p is None:
        p = np.ones(shape=(max_class, 1)) / max_class
        
    features_mean = features.mean(0)
    cov_w = np.zeros(shape=(m, m))
    cov_b = np.zeros(shape=(m, m))

    for k in range(max_class):
        ii = (norm == k)                                  
        class_features = features[ii,:]                    
        class_mean = class_features.mean(0)                
        class_cov = np.cov(class_features, rowvar=False)   
        
        cov_w += p[k] * class_cov                         
        
        dif = (class_mean - features_mean).reshape((m, 1))
        cov_b += p[k] * dif @ dif.T                 
    try:
        return np.trace(np.linalg.inv(cov_w) @ cov_b)
    except np.linalg.LinAlgError:
        return - np.inf

In [4]:
# Extraccion de caracteristicas
# Probamos con Gabor, Haralich, HOG, LBP, LBP por gama de colores y escala de grises

def imshow(image):
    pil_image = Image.fromarray(image)
    pil_image.show()

def get_image(path, show=False):
    img = cv2.imread(path)
    if show:
        imshow(img)
    return img

def dirfiles(img_path,img_ext):
    img_names = fnmatch.filter(sorted(os.listdir(img_path)),img_ext)
    return img_names


def num2fixstr(x,d):
    st = '%0*d' % (d,x)
    return st

def extract_features(dirpath,fmt):
    
    st = '*.'+fmt
    img_names = dirfiles(dirpath+'/',st)
    n = len(img_names)
    print(n)
    for i in range(n):
        img_path = img_names[i]
        img      = get_image(dirpath+'/'+img_path)
        escala_grises     = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        X_0      = lbp_features(escala_grises, hdiv=1, vdiv=1, mapping='nri_uniform')
        rojo     = lbp_features(img[:,:,0], hdiv=1, vdiv=1, mapping='nri_uniform')
        verde   = lbp_features(img[:,:,1], hdiv=1, vdiv=1, mapping='nri_uniform')
        azul    = lbp_features(img[:,:,2], hdiv=1, vdiv=1, mapping='nri_uniform')
        # Haralick = haralick_features(img.astype(int))
        # hog = hog_features(i, v_windows=3, h_windows=3, n_bins=8)
        features = np.asarray(np.concatenate((X_0, rojo, verde, azul)))
        
        if i==0:
            m = features.shape[0]
            data = np.zeros((n,m))
            print('size of extracted features:')
            print(features.shape)
        data[i]    = features
    return data


In [23]:
# Para aprender como usar SFS, vimos el codigo base de pybalu, pero reemplazamos todas las dependencias 
# dentro de la libreria por metodos propios.

def Sequential_Feature_Selector(features, classes, n_features):
    remaining_feats = set(np.arange(features.shape[1]))
    selected = list()
    curr_feats = np.zeros((features.shape[0], 0))
    options = dict()

    def fisher_score(i):
        feats = np.hstack([curr_feats, features[:, i].reshape(-1, 1)])
        m = features.shape[1]
        norm = classes.ravel() - classes.min()
        max_class = norm.max() + 1
        p = np.ones(shape=(max_class, 1)) / max_class

        features_mean = feats.mean(0)
        cov_w = np.zeros(shape=(m, m))
        cov_b = np.zeros(shape=(m, m))

        for k in range(max_class):
            ii = (norm == k)                                  
            class_features = features[ii,:]                    
            class_mean = class_features.mean(0)                
            class_cov = np.cov(class_features, rowvar=False)   

            cov_w += p[k] * class_cov                         

            dif = (class_mean - features_mean).reshape((m, 1))
            cov_b += p[k] * dif @ dif.T                 
        try:
            return np.trace(np.linalg.inv(cov_w) @ cov_b)
        except np.linalg.LinAlgError:
            return - np.inf
        

    _range = tqdm.trange(n_features, desc='', unit_scale=True, unit=' features')

    for _ in _range:
        new_selected = max(remaining_feats, key=fisher_score)
        selected.append(new_selected)
        remaining_feats.remove(new_selected)
        curr_feats = np.hstack([curr_feats, features[:, new_selected].reshape(-1, 1)])

    return np.array(selected)

In [15]:
# Si bien lo implementamos, no lo usaremos, ya que demostro no aportar a nuestro accuracy de prediccion
# El uso de numpy inspirado por https://sebastianraschka.com/Articles/2014_pca_step_by_step.html

def PCA(data, n_componentes):
    # seleccionadas = [columnas] #las columnas que queremos usar para el pca. Data debe ser un DF de pandas
    
    # x = data.loc[:, seleccionadas].values  Sin label !!!!!
    # y = data.loc[:, [labels]].values  
    
    # primero se saca la matriz de covarianza, usando la media y la desviacion estandar de los datos.
    # Luego sacamos los eigen vectors

    # Ordenamos los eigen vectors 
    pares = list(zip(np.linalg.eig(np.cov((x - x.mean()) / (x.std()).T) )[0], np.linalg.eig(np.cov((x - x.mean()) / (x.std()).T) )[1]))
    pares.sort(key=lambda vector: vector[0], reverse=True)
    vectores_seleccionados = []
    
    # sacamos los N primeros
    for component in range(columnas):
        vectores_seleccionados.append(pares[component][1])  
            
    matriz_scatter = np.hstack(map(lambda vector: vectores_seleccionados.reshape(len(seleccionadas), 1), vectores_seleccionados))
    return pd.concat([pd.DataFrame(data = (x - x.mean()) / (x.std()).dot(matriz_scatter), columns=[f'PC{componente}' for componente in range(1, n_componentes + 1)]), data[['char']]], axis=1)

In [16]:
X_Train_clean_walls = [picture for picture in listdir('./Training_0') if picture.endswith('png')]
x_Train_intervened_walls = [picture for picture in listdir('./Training_1') if picture.endswith('png')]
Y_Testing_clean_walls = [picture for picture in listdir('./Testing_0') if picture.endswith('png')]
y_Testing_intervened_walls = [picture for picture in listdir('./Testing_1') if picture.endswith('png')]

In [17]:
print("Clean Train: {0}, Intervened Train:{1}, Clean Test:{2}, Intervened Test:{3}".format(len(X_Train_clean_walls), 
                                                                                           len(x_Train_intervened_walls), 
                                                                                           len(Y_Testing_clean_walls), 
                                                                                           len(y_Testing_intervened_walls)))

Clean Train: 4000, Intervened Train:4000, Clean Test:1000, Intervened Test:1000


In [18]:
X0_train = extract_features('Training_0','png')
X1_train = extract_features('Training_1','png')
X0_test  = extract_features('Testing_0','png')
X1_test  = extract_features('Testing_1','png')

4000
size of extracted features:
(236,)
4000
size of extracted features:
(236,)
1000
size of extracted features:
(236,)
1000
size of extracted features:
(236,)


In [19]:
print('Training Subset:')
X_train  = np.concatenate((X0_train, X1_train),axis=0)
d0_train = np.zeros([X0_train.shape[0], 1],dtype=int)
d1_train = np.ones([X1_train.shape[0], 1],dtype=int)
d_train  = np.concatenate((d0_train, d1_train),axis=0)
print('Original extracted features: '+str(X_train.shape[1])+ '('+str(X_train.shape[0])+' samples)')

Training Subset:
Original extracted features: 236(8000 samples)


In [20]:
# Eliminamos caracteristicas altamente correlacionadas o constantes para set de Training
sclean = clean(X_train,show=True)
X_train_clean = X_train[:,sclean]
print('cleaned features: '+str(X_train_clean.shape[1])+ '('+str(X_train_clean.shape[0])+' samples)')

Clean: number of features reduced from 236 to 212.
cleaned features: 212(8000 samples)


In [21]:
# Normalizamos las columnas de datos
X_train_norm = X_train_clean * (1 / X_train_clean.std(0)) + (- X_train_clean.mean(0) / X_train_clean.std(0))

print('normalized features: '+str(X_train_norm.shape[1])+ '('+str(X_train_norm.shape[0])+' samples)')

normalized features: 212(8000 samples)


In [ ]:
# Training: Feature selection
    ssfs = Sequential_Feature_Selector(X_train_norm, d_train, n_features=80)
X_train_sfs = X_train_norm[:,ssfs]
print('selected features: '+str(X_train_sfs.shape[1])+ '('+str(X_train_sfs.shape[0])+' samples)')

In [ ]:
# Testing dataset
print('Testing Subset:')
X_test  = np.concatenate((X0_test,X1_test),axis=0)
d0_test = np.zeros([X0_test.shape[0],1],dtype=int)
d1_test = np.ones([X1_test.shape[0],1],dtype=int)
d_test  = np.concatenate((d0_test,d1_test),axis=0)

In [ ]:
# Testing: Cleaning
X_test_clean = X_test[:,sclean]

# Testing: Normalization
X_test_norm = X_test_clean * (1 / X_train_clean.std(0)) + (- X_train_clean.mean(0) / X_train_clean.std(0))

# Testing: Feature selection
X_test_sfs = X_test_norm[:,ssfs]
print('clean+norm+sfs features: '+str(X_test_sfs.shape[1])+ '('+str(X_test_sfs.shape[0])+' samples)')


In [ ]:
# Classification on Testing dataset
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_sfs, d_train)

acc = accuracy_score(d_test,knn.predict(X_test_sfs))
print('Confusion Matrix:')
print(confusion_matrix(d_test,knn.predict(X_test_sfs)) )
print('Accuracy = '+str(acc))
print()


plt.figure(figsize=(7,5))
heatmap(confusion_matrix(d_test,knn.predict(X_test_sfs)) , annot=True, fmt="d", cmap="YlGnBu")
plt.xlim(0,confusion_matrix(d_test,knn.predict(X_test_sfs)).shape[0])
plt.ylim(confusion_matrix(d_test,knn.predict(X_test_sfs)).shape[0],0)
plt.title('Confusion Matrix Testing',fontsize=14)
plt.show()


In [ ]:
pca_sk = PCA(n_components=19)
X_train_pcaed = pca_sk.fit_transform(X_train_sfs)
X_test_pcaed = pca_sk.transform(X_test_sfs)

In [ ]:
# Classification on Testing dataset
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_pcaed, d_train)
ds = knn.predict(X_test_pcaed)
Accuracy(ds, d_test,'Testing')

In [ ]:
100: 0.9555
85:  0.9605
81:  0.96
80:  0.9635
79:  0.9595
76:  0.9575
75:  0.9575
70:  0.957
60:  0.955
50:  0.957
30:  0.94
20:  0.935